# FLAT input data prepping

This code prepares the final input file to the FLAT model. 

**Original code:** [Konstantinos Pegios](https://github.com/kopegios) <br />
**Conceptualization & Methodological review :** [Alexandros Korkovelos](https://github.com/akorkovelos) & [Konstantinos Pegios](https://github.com/kopegios)<br />
**Updates, Modifications:** [Alexandros Korkovelos](https://github.com/akorkovelos)<br />
**Funding:** The World Bank (contract number: 7190531), [KTH](https://www.kth.se/en/itm/inst/energiteknik/forskning/desa/welcome-to-the-unit-of-energy-systems-analysis-kth-desa-1.197296)

In [8]:
#Import modules and libraries

import geopandas as gpd
import rasterio as rio
import pandas as pd
import fiona
import gdal
import osr
import ogr
import rasterio.mask
import time
import os
import ogr, gdal, osr, os
import numpy as np
import itertools
import re

from osgeo import gdal,ogr
import struct
import csv

import glob

from rasterio.warp import calculate_default_transform, reproject
from rasterio.enums import Resampling
from rasterstats import point_query, zonal_stats
from pyproj import Proj
from shapely.geometry import Point, Polygon

In [9]:
# Import basegrid map (in csv format) in the desired target resolution. 
path = r"C:\Benin\agrodem_preprocessing\Downscaling\Output_Data\Creating_data_qgis_FAO"
name_of_file = "base_grid_yams_1km.csv"

In [10]:
# Import csv as pandas dataframe
df = pd.read_csv(path + "\\" + name_of_file)
df.head()

,id,state,c_code,country,crop,harv_ar_ha,yield,prod_tonne,statear_ha,permtr_km,lon,lat
0,519,Banikoara,BEN,Benin,Yams,5195.017005,0.441577,2294.0,437979.859838,292.611412,2.395929,11.688117
1,520,Banikoara,BEN,Benin,Yams,5195.017005,0.441577,2294.0,437979.859838,292.611412,2.404912,11.688117
2,521,Banikoara,BEN,Benin,Yams,5195.017005,0.441577,2294.0,437979.859838,292.611412,2.413896,11.688117
3,526,Banikoara,BEN,Benin,Yams,5195.017005,0.441577,2294.0,437979.859838,292.611412,2.458811,11.688117
4,527,Banikoara,BEN,Benin,Yams,5195.017005,0.441577,2294.0,437979.859838,292.611412,2.467794,11.688117


In [11]:
df.rename(columns={'id': 'alloc_key'}, inplace=True)

In [12]:
#Function to change the order of columns in the dataframe 
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

In [13]:
# Allocate order in dataframe columns
df = change_column_order(df, 'alloc_key', 1)
df = change_column_order(df, 'lon', 2)
df = change_column_order(df, 'lat', 3)
df = change_column_order(df, 'country', 4)
df = change_column_order(df, 'c_code', 5)
df = change_column_order(df, 'state', 6)
df = change_column_order(df, 'statear_ha', 7)
df = change_column_order(df, 'permtr_km', 8)
df = change_column_order(df, 'crop', 9)
df = change_column_order(df, 'harv_ar_ha', 11)

In [14]:
df.head()

,alloc_key,lon,lat,country,c_code,state,statear_ha,permtr_km,crop,yield,prod_tonne,harv_ar_ha
0,519,2.395929,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,2294.0,5195.017005
1,520,2.404912,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,2294.0,5195.017005
2,521,2.413896,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,2294.0,5195.017005
3,526,2.458811,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,2294.0,5195.017005
4,527,2.467794,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,2294.0,5195.017005


In [15]:
# Add geometry and convert to spatial dataframe in source CRS
df['geometry'] = list(zip(df['lon'], df['lat']))
df['geometry'] = df['geometry'].apply(Point)
df = gpd.GeoDataFrame(df, geometry='geometry', crs={'init': 'epsg:4326'})

# Reproject data in to Ordnance Survey GB coordinates
df_osgb = df.to_crs({'init': 'epsg:3395'})

C:\Users\oluchi\.conda\envs\agrodem_run\lib\site-packages\pyproj\crs\crs.py:55: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [16]:
df_osgb.head()

,alloc_key,lon,lat,country,c_code,state,statear_ha,permtr_km,crop,yield,prod_tonne,harv_ar_ha,geometry
0,519,2.395929,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,2294.0,5195.017005,POINT (266713.620 1301583.755)
1,520,2.404912,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,2294.0,5195.017005,POINT (267713.620 1301583.755)
2,521,2.413896,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,2294.0,5195.017005,POINT (268713.620 1301583.755)
3,526,2.458811,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,2294.0,5195.017005,POINT (273713.620 1301583.755)
4,527,2.467794,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,2294.0,5195.017005,POINT (274713.620 1301583.755)


<div class="alert alert-block alert-warning">
<b>Note:</b> The geodataframe, contains crop points with needed attributes including a geometry column. It needs to be exported as shapefile. However, when this is done with geopandas, the column geometry (Point(x,y)) is not transfered in the attribute table. This is however needed for the extraction process below, otherwise "None" values are returned. In order to overcome this issue, the csv is imported into Qgis and then exported as feature layer in the same folder. This solves the issue. 
</div>

In [17]:
# Define output path
# Overwriting the csv file
#path = r"N:\Agrodem\Downscaling\Output_Data\FAO_AgroMap_Crops"
path = r"C:\Benin\agrodem_preprocessing\Downscaling\Output_Data\Crop_Maps_modified_FAO"
shpname = "Ben_Yams_2018_admin2_1km.csv"

#MAIZE
df_osgb.to_csv(os.path.join(path,"{c}".format(c=shpname)), index=False)
df_osgb.to_file('Ben_Yams_2018_admin2_1km.shp',driver = 'ESRI Shapefile')
# Define output path
#path = r"N:\Agrodem\Downscaling\Output_Data\Crop_Maps"
#path = r"C:\Benin\agrodem_preprocessing\Downscaling\Output_Data\Crop_Maps"

#Maize


#### >>> Please open Qgis and act as suggested above<<<

In [18]:
y = gpd.read_file(r'C:\Github\agrodem_new\agrodem_preprocessing\Downscaling\Downscaling_prepping_FAO\Ben_Yams_2018_admin2_1km.shp')

In [19]:
y.head()

,alloc_key,lon,lat,country,c_code,state,statear_ha,permtr_km,crop,yield,prod_tonne,harv_ar_ha,geometry
0,519,2.395929,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,2294.0,5195.017005,POINT (266713.620 1301583.755)
1,520,2.404912,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,2294.0,5195.017005,POINT (267713.620 1301583.755)
2,521,2.413896,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,2294.0,5195.017005,POINT (268713.620 1301583.755)
3,526,2.458811,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,2294.0,5195.017005,POINT (273713.620 1301583.755)
4,527,2.467794,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,2294.0,5195.017005,POINT (274713.620 1301583.755)


## Extracting raster input to csv 

In [27]:
#raster_path = r"N:\Agrodem\Downscaling\Input_Data\rasters"
raster_path = r"C:\Oluchi\Downscaling\Input_Data\raster_reprojected"
raster_files =[]
#csvoutpath = r"N:\Agrodem\Downscaling\Output_Data\FLAT_input\1km_Rice"
csvoutpath = r"C:\Oluchi\Downscaling\Output_Data\FLAT_Input\FAO\1km_Yams"

print ("Reading independent variables...")

for i in os.listdir(raster_path):
    if i.endswith('.tif'):
        raster_files.append(i)  

#shp_filename = r"N:\Agrodem\Downscaling\Output_Data\FAO_AgroMap_Crops\shapefiles\Moz_Rice_Harv_2000_admin2_1km.shp"
shp_filename = r'C:\Github\agrodem_new\agrodem_preprocessing\Downscaling\Downscaling_prepping_FAO\Ben_Yams_2018_admin2_1km.shp'
print ("Extracting raster values to points...")

for i in raster_files:
    print("Extracting " + i + " values...")
    src_filename = raster_path + "\\" + i 
    li_values = list()

    src_ds=gdal.Open(src_filename) 
    gt=src_ds.GetGeoTransform()
    rb=src_ds.GetRasterBand(1)
    ds=ogr.Open(shp_filename)    
    lyr=ds.GetLayer()
    
    for feat in lyr:
        geom = feat.GetGeometryRef()
        feat_id = feat.GetField('alloc_key')
        mx,my=geom.GetX(), geom.GetY()  #coord in map units

        #Convert from map to pixel coordinates.
        #Only works for geotransforms with no rotation.
        px = int((mx - gt[0]) / gt[1]) #x pixel
        py = int((my - gt[3]) / gt[5]) #y pixel

        intval=rb.ReadAsArray(px,py,1,1)
        li_values.append([feat_id, intval[0]])
        
    print ("Writing " + i + " values to csv...")
    
    with open(csvoutpath + "\\" + i.split('.')[0] + '.csv', 'w') as csvfile:
        wr = csv.writer(csvfile)
        wr.writerows(li_values)

Reading independent variables...
Extracting raster values to points...
Extracting BDRICM.tif values...
Writing BDRICM.tif values to csv...
Extracting BLD.tif values...
Writing BLD.tif values to csv...
Extracting CLYPPT.tif values...
Writing CLYPPT.tif values to csv...
Extracting DRAINFAO.tif values...
Writing DRAINFAO.tif values to csv...
Extracting GFSAD-landcover.tif values...
Writing GFSAD-landcover.tif values to csv...
Extracting landcover.tif values...
Writing landcover.tif values to csv...
Extracting MODIS-Band2.tif values...
Writing MODIS-Band2.tif values to csv...
Extracting modis-evi.tif values...
Writing modis-evi.tif values to csv...
Extracting modis-ndvi.tif values...
Writing modis-ndvi.tif values to csv...
Extracting ORCDRC.tif values...
Writing ORCDRC.tif values to csv...
Extracting PHIHOX.tif values...
Writing PHIHOX.tif values to csv...
Extracting prec.tif values...
Writing prec.tif values to csv...
Extracting srad.tif values...
Writing srad.tif values to csv...
Extract

## Merge csv files with crop

In [28]:
#Import  previously extracted csv file (modified version of the basegrid map) 

path = r"C:\Benin\agrodem_preprocessing\Downscaling\Output_Data\Crop_Maps_modified_FAO"
name_of_file= "Ben_Yams_2018_admin2_1km.csv"

flat_input = pd.read_csv(path + "\\" + name_of_file)

In [29]:
#csvoutpath = r"N:\Agrodem\Downscaling\Output_Data\FLAT_input\1km_Rice"
csvoutpath = r"C:\Oluchi\Downscaling\Output_Data\FLAT_Input\FAO\1km_Yams"
csv_files = []

print ("Reading csv files...")

for i in os.listdir(csvoutpath):
    if i.endswith('.csv'):
        csv_files.append(i)  

for i in csv_files:
    print('Reading...'+ i)    
    df_csv = pd.read_csv(csvoutpath + "//" + i, index_col=None, header=None)
    df_csv.iloc[:,1] = df_csv.iloc[:,1].astype(str)
    df_csv.iloc[:,1] = df_csv.iloc[:,1].str.replace('[','')
    df_csv.iloc[:,1] = df_csv.iloc[:,1].str.replace(']','')
    columnName = i.split('.')[0]
    print("Merging..." + columnName)
    flat_input[columnName] = df_csv.iloc[:,1]

Reading csv files...
Reading...BDRICM.csv
Merging...BDRICM
Reading...BLD.csv
Merging...BLD
Reading...CLYPPT.csv
Merging...CLYPPT
Reading...DRAINFAO.csv
Merging...DRAINFAO
Reading...GFSAD-landcover.csv
Merging...GFSAD-landcover
Reading...landcover.csv
Merging...landcover
Reading...MODIS-Band2 (1).csv
Merging...MODIS-Band2 (1)
Reading...MODIS-Band2.csv
Merging...MODIS-Band2
Reading...modis-evi.csv
Merging...modis-evi
Reading...modis-ndvi.csv
Merging...modis-ndvi
Reading...ORCDRC.csv
Merging...ORCDRC
Reading...PHIHOX.csv
Merging...PHIHOX
Reading...prec.csv
Merging...prec
Reading...srad.csv
Merging...srad
Reading...SRTM-elevation.csv
Merging...SRTM-elevation
Reading...tavg.csv
Merging...tavg
Reading...TEXMHT.csv
Merging...TEXMHT
Reading...wind.csv
Merging...wind


## Cleaning and normalizing dataframe

In [30]:
#Show columns
flat_input.columns

Index(['alloc_key', 'lon', 'lat', 'country', 'c_code', 'state', 'statear_ha',
       'permtr_km', 'crop', 'yield', 'prod_tonne', 'harv_ar_ha', 'geometry',
       'BDRICM', 'BLD', 'CLYPPT', 'DRAINFAO', 'GFSAD-landcover', 'landcover',
       'MODIS-Band2 (1)', 'MODIS-Band2', 'modis-evi', 'modis-ndvi', 'ORCDRC',
       'PHIHOX', 'prec', 'srad', 'SRTM-elevation', 'tavg', 'TEXMHT', 'wind'],
      dtype='object')

In [31]:

flat_input.head(10)

,alloc_key,lon,lat,country,c_code,state,statear_ha,permtr_km,crop,yield,...,modis-evi,modis-ndvi,ORCDRC,PHIHOX,prec,srad,SRTM-elevation,tavg,TEXMHT,wind
0,519,2.395929,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,...,2519.,4631.,10.,67.,69.916664,19623.916,266,28.041666,7.,2.9
1,520,2.404912,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,...,2382.,4737.,9.,65.,69.916664,19623.916,269,28.041666,7.,2.9
2,521,2.413896,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,...,2759.,5143.,10.,64.,69.916664,19630.75,271,28.025002,7.,2.9
3,526,2.458811,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,...,2476.,4554.,10.,64.,70.333336,19627.,313,27.983337,7.,3.
4,527,2.467794,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,...,2657.,4986.,10.,64.,70.25,19610.166,307,28.000002,7.,2.8
5,528,2.476778,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,...,2525.,4547.,8.,64.,70.25,19610.166,307,28.000002,9.,2.8
6,529,2.485761,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,...,2322.,3750.,8.,64.,70.166664,19601.166,296,28.025002,9.,2.7
7,530,2.494744,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,...,2317.,3617.,9.,64.,70.25,19607.,291,28.000002,9.,2.9
8,531,2.503727,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,...,2427.,3758.,9.,64.,70.25,19607.,287,28.000002,9.,2.9
9,851,2.315081,11.679263,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,...,2519.,4542.,10.,66.,69.833336,19611.834,252,28.175001,7.,3.


In [32]:
#Delete geometry column
flat_input = flat_input.drop(["geometry"], axis=1)

#### Convert values in newly added columns into float 

In [34]:
flat_input["harv_ar_ha"] = flat_input["harv_ar_ha"].astype(float)
flat_input["BDRICM"] = flat_input["BDRICM"].astype(float)
flat_input["BLD"] = flat_input["BLD"].astype(float)
flat_input["CLYPPT"] = flat_input["CLYPPT"].astype(float)
flat_input["DRAINFAO"] = flat_input["DRAINFAO"].astype(float)
flat_input["GFSAD-landcover"] = flat_input["GFSAD-landcover"].astype(float)
flat_input["landcover"] = flat_input["landcover"].astype(float)
flat_input["MODIS-Band2"] = flat_input["MODIS-Band2"].astype(float)
flat_input["modis-evi"] = flat_input["modis-evi"].astype(float)
flat_input["modis-ndvi"] = flat_input["modis-ndvi"].astype(float)
flat_input["ORCDRC"] = flat_input["ORCDRC"].astype(float)
flat_input["PHIHOX"] = flat_input["PHIHOX"].astype(float)
flat_input["prec"] = flat_input["prec"].astype(float)
flat_input["srad"] = flat_input["srad"].astype(float)
flat_input["SRTM-elevation"] = flat_input["SRTM-elevation"].astype(float)
flat_input["tavg"] = flat_input["tavg"].astype(float)
flat_input["TEXMHT"] = flat_input["TEXMHT"].astype(float)
flat_input["wind"] = flat_input["wind"].astype(float)

In [35]:
flat_input.dtypes

alloc_key            int64
lon                float64
lat                float64
country             object
c_code              object
state               object
statear_ha         float64
permtr_km          float64
crop                object
yield              float64
prod_tonne         float64
harv_ar_ha         float64
BDRICM             float64
BLD                float64
CLYPPT             float64
DRAINFAO           float64
GFSAD-landcover    float64
landcover          float64
MODIS-Band2 (1)     object
MODIS-Band2        float64
modis-evi          float64
modis-ndvi         float64
ORCDRC             float64
PHIHOX             float64
prec               float64
srad               float64
SRTM-elevation     float64
tavg               float64
TEXMHT             float64
wind               float64
dtype: object

#### Normalize EVI and NDVI 

In [36]:
flat_input['modis-evi'] = flat_input['modis-evi']*0.0001
flat_input['modis-ndvi'] = flat_input['modis-ndvi']*0.0001

In [37]:
flat_input.head()

,alloc_key,lon,lat,country,c_code,state,statear_ha,permtr_km,crop,yield,...,modis-evi,modis-ndvi,ORCDRC,PHIHOX,prec,srad,SRTM-elevation,tavg,TEXMHT,wind
0,519,2.395929,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,...,0.2519,0.4631,10.0,67.0,69.916664,19623.916,266.0,28.041666,7.0,2.9
1,520,2.404912,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,...,0.2382,0.4737,9.0,65.0,69.916664,19623.916,269.0,28.041666,7.0,2.9
2,521,2.413896,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,...,0.2759,0.5143,10.0,64.0,69.916664,19630.750,271.0,28.025002,7.0,2.9
3,526,2.458811,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,...,0.2476,0.4554,10.0,64.0,70.333336,19627.000,313.0,27.983337,7.0,3.0
4,527,2.467794,11.688117,Benin,BEN,Banikoara,437979.859838,292.611412,Yams,0.441577,...,0.2657,0.4986,10.0,64.0,70.250000,19610.166,307.0,28.000002,7.0,2.8


#### Drop  areas that are indicated as non-cropland in Global Food Security-support Analysis Data at 30m(GFSAD30) [here](https://developers.google.com/earth-engine/datasets/catalog/USGS_GFSAD1000_V0#bands)

In [38]:
flat_input.drop(flat_input[flat_input["GFSAD-landcover"] == 0].index, inplace=True)
#flat_input = flat_input[flat_input["Landcover"] == ]
#flat_input = flat_input.drop(["Landcover"], axis=1)
flat_input["GFSAD-landcover"]

62       8.0
63       8.0
64       8.0
65       8.0
66       8.0
        ... 
73786    8.0
73799    8.0
73800    8.0
73801    8.0
73802    8.0
Name: GFSAD-landcover, Length: 36568, dtype: float64

#### Fixing out-of-range values

In [39]:
# remove blank spaces in state names    
flat_input["state"].replace('\s+', '_',regex=True,inplace=True)
flat_input["state"].replace("'", '_',regex=True,inplace=True)

In [40]:
# Re-indexing allocation keys to avoid dublicates
flat_input = flat_input.assign(alloc_key=np.arange(len(flat_input))).reset_index(drop=True)

In [41]:
# Turning NaN rows to 0
flat_input.fillna(0,inplace=True)

In [42]:
# Turn all negative values to 0
pred_columns = ['tavg', 'srad', 'prec', 'wind', 'PHIHOX',
                'BDRICM', 'BLD', 'CLYPPT', 'TEXMHT', 'ORCDRC',
                'DRAINFAO', 'MODIS-Band2', 'SRTM-elevation']

for col in pred_columns:
    mvalue = flat_input[col].mean()
    print (mvalue)
    flat_input[col][flat_input[col]<0] = 0

27.44860515015861
17808.23946081273
84.54652974928901
2.204621527018158
62.453647998249835
137.58769962809012
1367.4879676219646
18.175618026690003
8.131070881645154
12.242971997374754
3.557974185079851
3336.4792773542986
243.7850853204988


C:\Users\oluchi\.conda\envs\agrodem_run\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [43]:
for col in pred_columns:
    mvalue = flat_input[col].mean()
    print (mvalue)

27.44860515015861
17808.23946081273
84.54652974928901
2.204621527018158
62.453647998249835
137.58769962809012
1367.4879676219646
18.175618026690003
8.131070881645154
12.242971997374754
3.557974185079851
3336.4792773542986
243.7850853204988


In [44]:
flat_input.columns

Index(['alloc_key', 'lon', 'lat', 'country', 'c_code', 'state', 'statear_ha',
       'permtr_km', 'crop', 'yield', 'prod_tonne', 'harv_ar_ha', 'BDRICM',
       'BLD', 'CLYPPT', 'DRAINFAO', 'GFSAD-landcover', 'landcover',
       'MODIS-Band2 (1)', 'MODIS-Band2', 'modis-evi', 'modis-ndvi', 'ORCDRC',
       'PHIHOX', 'prec', 'srad', 'SRTM-elevation', 'tavg', 'TEXMHT', 'wind'],
      dtype='object')

In [45]:
flat_input = change_column_order(flat_input, 'alloc_key', 0)
flat_input = change_column_order(flat_input, 'state', 1)
flat_input = change_column_order(flat_input, 'lon', 2)
flat_input = change_column_order(flat_input, 'lat', 3)
flat_input = change_column_order(flat_input, 'country', 4)
flat_input = change_column_order(flat_input, 'c_code', 5)

flat_input = change_column_order(flat_input, 'statear_ha', 6)
flat_input = change_column_order(flat_input, 'permtr_km', 7)
flat_input = change_column_order(flat_input, 'crop', 8)
flat_input = change_column_order(flat_input, 'harv_ar_ha', 9)
flat_input = change_column_order(flat_input, 'yield', 10)
flat_input = change_column_order(flat_input, 'prod_tonne', 11)

flat_input = change_column_order(flat_input, 'tavg', 13)
flat_input = change_column_order(flat_input, 'srad', 14)
flat_input = change_column_order(flat_input, 'prec', 15)
flat_input = change_column_order(flat_input, 'wind', 16)

flat_input = change_column_order(flat_input, 'PHIHOX', 17)
flat_input = change_column_order(flat_input, 'BDRICM', 18)
flat_input = change_column_order(flat_input, 'BLD', 19)
flat_input = change_column_order(flat_input, 'CLYPPT', 20)
flat_input = change_column_order(flat_input, 'TEXMHT', 21)
flat_input = change_column_order(flat_input, 'ORCDRC', 22)
flat_input = change_column_order(flat_input, 'DRAINFAO', 23)

flat_input = change_column_order(flat_input, 'modis-evi', 24)
flat_input = change_column_order(flat_input, 'modis-ndvi', 25)

flat_input = change_column_order(flat_input, 'MODIS-Band2', 26)
flat_input = change_column_order(flat_input, 'SRTM-elevation', 27)
flat_input = change_column_order(flat_input, 'landcover', 28)
flat_input = change_column_order(flat_input, 'GFSAD-landcover', 28)

In [46]:
flat_input.head()

,alloc_key,state,lon,lat,country,c_code,statear_ha,permtr_km,crop,harv_ar_ha,...,TEXMHT,ORCDRC,DRAINFAO,modis-evi,modis-ndvi,MODIS-Band2,SRTM-elevation,landcover,GFSAD-landcover,MODIS-Band2 (1)
0,0,Banikoara,2.342030,11.661554,Benin,BEN,437979.859838,292.611412,Yams,5195.017005,...,4.0,13.0,2.0,0.2292,0.4461,3021.500000,251.0,9.0,8.0,3021.5
1,1,Banikoara,2.351013,11.661554,Benin,BEN,437979.859838,292.611412,Yams,5195.017005,...,7.0,13.0,2.0,0.2714,0.4683,3239.666667,251.0,9.0,8.0,3239.66666667
2,2,Banikoara,2.359997,11.661554,Benin,BEN,437979.859838,292.611412,Yams,5195.017005,...,7.0,9.0,2.0,0.2656,0.4058,3343.900000,253.0,9.0,8.0,3343.9
3,3,Banikoara,2.368980,11.661554,Benin,BEN,437979.859838,292.611412,Yams,5195.017005,...,7.0,9.0,3.0,0.2118,0.3461,3250.666667,258.0,9.0,8.0,3250.66666667
4,4,Banikoara,2.377963,11.661554,Benin,BEN,437979.859838,292.611412,Yams,5195.017005,...,7.0,9.0,3.0,0.1940,0.3116,3230.454545,265.0,9.0,8.0,3230.45454545


In [47]:
flat_input.columns

Index(['alloc_key', 'state', 'lon', 'lat', 'country', 'c_code', 'statear_ha',
       'permtr_km', 'crop', 'harv_ar_ha', 'yield', 'prod_tonne', 'tavg',
       'srad', 'prec', 'wind', 'PHIHOX', 'BDRICM', 'BLD', 'CLYPPT', 'TEXMHT',
       'ORCDRC', 'DRAINFAO', 'modis-evi', 'modis-ndvi', 'MODIS-Band2',
       'SRTM-elevation', 'landcover', 'GFSAD-landcover', 'MODIS-Band2 (1)'],
      dtype='object')

#### Final column fixing

In [48]:
# give crop name
crop_modelled ="Yams"

# Dropping columns
flat_input = flat_input.drop(["country"], axis=1)
flat_input = flat_input.drop(["c_code"], axis=1)
flat_input = flat_input.drop(["permtr_km"], axis=1)
flat_input = flat_input.drop(["crop"], axis=1)
flat_input = flat_input.drop(["yield"], axis=1)
flat_input = flat_input.drop(["prod_tonne"], axis=1)
flat_input = flat_input.drop(["landcover"], axis=1) 
 


#Remaning columns
flat_input.rename(columns={'state': 'NAME'}, inplace=True)
flat_input.rename(columns={'statearea_ha': 'statearea'}, inplace=True)
flat_input.rename(columns={'harv_ar_ha': crop_modelled}, inplace=True)


In [49]:
flat_input

,alloc_key,NAME,lon,lat,statear_ha,Yams,tavg,srad,prec,wind,...,CLYPPT,TEXMHT,ORCDRC,DRAINFAO,modis-evi,modis-ndvi,MODIS-Band2,SRTM-elevation,GFSAD-landcover,MODIS-Band2 (1)
0,0,Banikoara,2.342030,11.661554,437979.859838,5195.017005,28.074999,19546.166,70.666664,2.9,...,30.0,4.0,13.0,2.0,0.2292,0.4461,3021.500000,251.0,8.0,3021.5
1,1,Banikoara,2.351013,11.661554,437979.859838,5195.017005,28.066666,19547.500,70.583336,2.9,...,26.0,7.0,13.0,2.0,0.2714,0.4683,3239.666667,251.0,8.0,3239.66666667
2,2,Banikoara,2.359997,11.661554,437979.859838,5195.017005,28.041666,19522.250,70.833336,3.0,...,20.0,7.0,9.0,2.0,0.2656,0.4058,3343.900000,253.0,8.0,3343.9
3,3,Banikoara,2.368980,11.661554,437979.859838,5195.017005,28.066666,19526.584,70.750000,2.9,...,22.0,7.0,9.0,3.0,0.2118,0.3461,3250.666667,258.0,8.0,3250.66666667
4,4,Banikoara,2.377963,11.661554,437979.859838,5195.017005,28.066666,19526.584,70.750000,2.9,...,22.0,7.0,9.0,3.0,0.1940,0.3116,3230.454545,265.0,8.0,3230.45454545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36563,36563,Bohicon,2.198300,7.122096,15732.941088,3.216729,27.583334,16028.417,85.083336,1.3,...,18.0,9.0,16.0,5.0,0.3799,0.5773,3971.175000,56.0,8.0,3971.175
36564,36564,Bohicon,2.171350,7.131069,15732.941088,3.216729,27.483332,15974.833,85.833336,1.5,...,20.0,9.0,16.0,5.0,0.3806,0.5935,3974.611111,66.0,8.0,3974.61111111
36565,36565,Bohicon,2.180334,7.131069,15732.941088,3.216729,27.483332,15974.833,85.833336,1.5,...,18.0,9.0,16.0,5.0,0.3823,0.5838,3937.000000,77.0,8.0,3937.
36566,36566,Bohicon,2.189317,7.131069,15732.941088,3.216729,27.516668,16063.250,85.166664,1.3,...,19.0,9.0,17.0,5.0,0.3702,0.5785,3866.200000,70.0,8.0,3866.2


In [50]:
flat_input.columns

Index(['alloc_key', 'NAME', 'lon', 'lat', 'statear_ha', 'Yams', 'tavg', 'srad',
       'prec', 'wind', 'PHIHOX', 'BDRICM', 'BLD', 'CLYPPT', 'TEXMHT', 'ORCDRC',
       'DRAINFAO', 'modis-evi', 'modis-ndvi', 'MODIS-Band2', 'SRTM-elevation',
       'GFSAD-landcover', 'MODIS-Band2 (1)'],
      dtype='object')

In [51]:
# This part prints full results

#path = r"N:\Agrodem\Downscaling\Output_Data\FLAT_input"
path =r"C:\Oluchi\Downscaling\Output_Data\FLAT_Input\FAO"
name_of_flat_input_file = "flat_input_Yams_1km"

flat_input.to_csv(os.path.join(path,"{c}.csv".format(c=name_of_flat_input_file)), index=False)